In [11]:
import tensorflow as tf
from tensorflow import keras
from keras import utils
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Input, Dense, Flatten, Concatenate

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
from IPython.display import Image

import os

## 데이터셋 다운로드 및 전처리

In [12]:
fashion_mnist = keras.datasets.fashion_mnist
((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()

In [13]:
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000, 10)
(10000, 28, 28)
(10000, 10)


## 모델 설계

In [33]:
# Create the base model from the pre-trained model Resnet-50
# ResNet50 불러오기에서 include_top=을 True가 아닌 False로 둠으로써 사전학습된 모델의 최상층 분류기를 사용하지 않겠다고 설정
# include_top을 True로 설정하면 크기는 224,224,3으로 제한

IMG_SHAPE = (32, 32, 3)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [34]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [17]:
len(base_model.layers)

175

In [18]:
np.random.seed(0)

sample_arr = [True, False]
bool_arr = np.random.choice(sample_arr, size=len(base_model.layers))
print('Numpy Array: ')
print(bool_arr)
print(len(bool_arr))

Numpy Array: 
[ True False False  True False False False False False False False  True
  True False  True  True  True  True  True False  True False False  True
  True False False False False  True False  True False  True False False
  True False False  True  True False  True False False False False False
  True False  True False False False False  True False  True  True False
 False  True False  True False  True  True  True  True  True False False
  True  True  True False False  True False  True  True False  True False
 False False False False False  True False False  True  True False  True
  True False False  True False  True  True False  True  True  True False
 False  True False  True  True  True  True  True False  True False  True
 False False False False False  True False False False False  True False
 False  True  True False  True  True  True  True False False  True  True
 False  True False False False False  True  True  True False  True False
 False False  True False  True  True 

In [23]:
base_model.trainable = True		# resnet 모델 학습동결을 해제한다
for idx, i in enumerate(base_model.layers):	# 143층부터의 학습은 해제상태로 두고, 
  i.trainable = bool_arr[idx]				# 이전까지의 학습은 동결한다.
  
for i in base_model.layers:	# 동결이 제대로 해제됐는지 약간 이전층부터 출력해본다.
  print(i.name, ':', i.trainable)

input_2 : True
conv1_pad : False
conv1_conv : False
conv1_bn : True
conv1_relu : False
pool1_pad : False
pool1_pool : False
conv2_block1_1_conv : False
conv2_block1_1_bn : False
conv2_block1_1_relu : False
conv2_block1_2_conv : False
conv2_block1_2_bn : True
conv2_block1_2_relu : True
conv2_block1_0_conv : False
conv2_block1_3_conv : True
conv2_block1_0_bn : True
conv2_block1_3_bn : True
conv2_block1_add : True
conv2_block1_out : True
conv2_block2_1_conv : False
conv2_block2_1_bn : True
conv2_block2_1_relu : False
conv2_block2_2_conv : False
conv2_block2_2_bn : True
conv2_block2_2_relu : True
conv2_block2_3_conv : False
conv2_block2_3_bn : False
conv2_block2_add : False
conv2_block2_out : False
conv2_block3_1_conv : True
conv2_block3_1_bn : False
conv2_block3_1_relu : True
conv2_block3_2_conv : False
conv2_block3_2_bn : True
conv2_block3_2_relu : False
conv2_block3_3_conv : False
conv2_block3_3_bn : True
conv2_block3_add : False
conv2_block3_out : False
conv3_block1_1_conv : True
conv3

In [28]:
inputs = Input((28, 28, 1))
resized_x = tf.keras.layers.experimental.preprocessing.Resizing(32, 32)(inputs)
first_conv_layer = Conv2D(3, 1, padding='same', activation=None)(resized_x)

x = base_model(first_conv_layer, training = False)
x = Flatten()(x)
outputs = Dense(10, activation = 'softmax')(x)

model = tf.keras.Model(inputs, outputs, name="fashion_mnist_resnet50_model")

In [25]:
model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate= 0.0001), 
              metrics=['accuracy'])

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.summary()

# model_res.summary() 결과, 맨 아래쪽 Trainable params의 수치가 눈에 띄게 늘어난 것을 볼 수 있다.

Model: "fashion_mnist_resnet50_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 resizing (Resizing)         (None, 32, 32, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 3)         6         
                                                                 
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
Total params: 23,608,208
Trainable par

In [26]:
hist_fine = model.fit(x_train, y_train,
                      validation_data = (x_test, y_test),
                      epochs = 20,
                      batch_size = 256,
                      verbose = 1,
                      callbacks=[early])

Epoch 1/20
235/235 [==============================] - 39s 80ms/step - loss: 0.9528 - accuracy: 0.6403 - val_loss: 0.6812 - val_accuracy: 0.7340
Epoch 2/20
235/235 [==============================] - 16s 67ms/step - loss: 0.5355 - accuracy: 0.7948 - val_loss: 0.4937 - val_accuracy: 0.8120
Epoch 3/20
235/235 [==============================] - 16s 67ms/step - loss: 0.4613 - accuracy: 0.8261 - val_loss: 0.5398 - val_accuracy: 0.8001
Epoch 4/20
235/235 [==============================] - 16s 67ms/step - loss: 0.4217 - accuracy: 0.8426 - val_loss: 0.4590 - val_accuracy: 0.8274
Epoch 5/20
235/235 [==============================] - 16s 67ms/step - loss: 0.3861 - accuracy: 0.8561 - val_loss: 0.4443 - val_accuracy: 0.8327
Epoch 6/20
235/235 [==============================] - 16s 67ms/step - loss: 0.3754 - accuracy: 0.8588 - val_loss: 0.4274 - val_accuracy: 0.8444
Epoch 7/20
235/235 [==============================] - 16s 67ms/step - loss: 0.3530 - accuracy: 0.8683 - val_loss: 0.3610 - val_accuracy:

In [54]:
class Random_Finetune_ResNet50():
    def __init__(self, input_shape):
        self.fitness = 0
        
        IMG_SHAPE = input_shape + (3,)
        self.base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
        sample_arr = [True, False]
        bool_arr = np.random.choice(sample_arr, size=len(self.base_model.layers))
        self.base_model.trainable = True
        for idx, i in enumerate(self.base_model.layers):
            i.trainable = bool_arr[idx]
        
    def forward(self, learning_rate=0.001):
        inputs = Input((28, 28, 1))
        resized_x = tf.keras.layers.experimental.preprocessing.Resizing(32, 32)(inputs)
        first_conv_layer = Conv2D(3, 1, padding='same', activation=None)(resized_x)

        x = self.base_model(first_conv_layer, training = False)
        x = Flatten()(x)
        outputs = Dense(10, activation = 'softmax')(x)

        model = tf.keras.Model(inputs, outputs, name="fashion_mnist_resnet50_model")

        # 'categorical_crossentropy'은 y[0]=[0, 0, 0, 0, 0, 0, 0, 0, 1], y[1, 0, 0, 0, 0, 0, 0, 0, 0]과 같이 one-hot-encoding label일 경우에 사용
        model.compile(loss="categorical_crossentropy", 
        optimizer=tf.keras.optimizers.Adam(learning_rate= learning_rate), 
        metrics=['accuracy'])
        
        return model
    
    def train_model(self, model, train_data, train_targets, validation_data=(x_test, y_test), epochs=20, batch_size=256):
    
        early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
        history = model.fit(train_data, train_targets,
                        validation_data = validation_data,
                        epochs = epochs,
                        batch_size = batch_size,
                        verbose = 1,
                        callbacks=[early])
        return history

In [ ]:
def train_model(model, train_data, train_targets, validation_data=(x_test, y_test), epochs=20, batch_size=256):
    
    early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    history = model.fit(train_data, train_targets,
                      validation_data = validation_data,
                      epochs = epochs,
                      batch_size = batch_size,
                      verbose = 1,
                      callbacks=[early])
    return history

In [53]:
N_POPULATION = 10
N_BEST = 5
N_CHILDREN = 5
PROB_MUTATION = 0.2

In [ ]:
i = 1
n_gen = 1
gene = Random_Finetune_ResNet50((32,32))
model = gene.forward()
history = gene.train_model(model, x_train, y_train, (x_test, y_test), 20, 256)
fitness = history.history['val_accuracy']
score = history.history['val_loss']
gene.fitness = fitness
print('Generation #%s, Genome #%s, Fitness: %s, Score: %s' % (n_gen, i, fitness, score))


In [37]:
genomes = [Random_Finetune_ResNet50((32,32)) for _ in range(N_POPULATION)]

In [40]:
print(len(genomes))
print(genomes[0])

10


In [42]:
def train_model(model, train_data, train_targets, validation_data=(x_test, y_test), epochs=20, batch_size=256):
    
    early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    history = model.fit(train_data, train_targets,
                      validation_data = validation_data,
                      epochs = epochs,
                      batch_size = batch_size,
                      verbose = 1,
                      callbacks=[early])
    return history

In [55]:
n_gen = 1
gene = Random_Finetune_ResNet50((32,32))
model = gene.forward()
history = train_model(model, x_train, y_train, (x_test, y_test), 20, 256)
fitness = history.history['val_accuracy']
score = history.history['val_loss']
gene.fitness = fitness
print('Generation #%s, Genome #%s, Fitness: %s, Score: %s' % (n_gen, i, fitness, score))

Epoch 1/20
235/235 [==============================] - 23s 74ms/step - loss: 2.3823 - accuracy: 0.1005 - val_loss: 2.3046 - val_accuracy: 0.1000
Epoch 2/20
235/235 [==============================] - 16s 68ms/step - loss: 2.3033 - accuracy: 0.0984 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/20
133/235 [===============>..............] - ETA: 6s - loss: 2.3026 - accuracy: 0.0982

KeyboardInterrupt: 

In [51]:
fitness.sort()
print(fitness)

[0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612]


In [43]:
best_genomes = None

In [44]:
n_gen = 0
while True:
  n_gen += 1

  for i, genome in enumerate(genomes):
      compile_model(genome, learning_rate=0.001)
      history = train_model(genome, x_train, y_train)
      fitness = history.history['val_accuracy']
      score = history.history['val_loss']
      genome.fitness = fitness

      print('Generation #%s, Genome #%s, Fitness: %s, Score: %s' % (n_gen, i, fitness, score))
  if best_genomes is not None:
      genomes.extend(best_genomes)
  genomes.sort(key=lambda x: x.fitness, reverse=True)

AttributeError: 'Random_Finetune_ResNet50' object has no attribute 'compile'